# 🚀 Пример использования LinearLongitudinalF16

Этот notebook демонстрирует работу с продольной динамической моделью самолёта F-16 в библиотеке TensorAeroSpace.

## 📋 Что мы изучим:
- Создание и настройка среды F-16
- Генерация опорных сигналов
- Выполнение шагов симуляции
- Анализ результатов

---

## 📦 Импорт необходимых библиотек

Загружаем все необходимые модули для работы с моделью F-16:

In [1]:
# Основные библиотеки
import gymnasium as gym 
import numpy as np
from tqdm import tqdm

# Модули TensorAeroSpace
from tensoraerospace.envs import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step
from tensoraerospace.agent.ihdp.model import IHDPAgent

## ⚙️ Настройка параметров симуляции

Определяем временные параметры и создаём опорный сигнал для управления углом тангажа:

In [2]:
# Параметры времени
dt = 0.01  # Шаг дискретизации (секунды)
tp = generate_time_period(tn=20, dt=dt)  # Временной период 20 секунд
tps = convert_tp_to_sec_tp(tp, dt=dt)  # Преобразование в секунды
number_time_steps = len(tp)  # Общее количество временных шагов

# Создание опорного сигнала
reference_signals = unit_step(degree=5, tp=tp, time_step=10, output_rad=True)
reference_signals = np.reshape(reference_signals, [1, -1])  # Формат: [1, время] для отслеживания угла тангажа

print(f'📊 Параметры симуляции:')
print(f'   • Шаг времени: {dt} сек')
print(f'   • Общее время: {tp[-1]:.1f} сек')
print(f'   • Количество шагов: {number_time_steps}')
print(f'   • Размер опорного сигнала: {reference_signals.shape}')

## 🛩️ Создание среды F-16

Создаём среду продольной динамики F-16 с 5 состояниями:
- **theta** - угол тангажа
- **alpha** - угол атаки
- **q** - угловая скорость тангажа
- **altitude** - высота
- **velocity** - скорость

In [3]:
# Создание среды F-16 с полной конфигурацией
env = gym.make('LinearLongitudinalF16-v0',
                number_time_steps=number_time_steps,  # Количество временных шагов
                initial_state=[[0],[0],[0],[0],[0]],  # Начальное состояние (все нули)
                reference_signal=reference_signals,   # Опорный сигнал
                state_space=["theta", "alpha", "q", "altitude", "velocity"],  # Пространство состояний
                output_space=["theta", "alpha", "q", "altitude", "velocity"], # Выходное пространство
                tracking_states=["theta"],           # Отслеживаемые состояния
                use_reward=False)                    # Без функции награды

# Инициализация среды
initial_observation, info = env.reset()
print(f'✅ Среда F-16 успешно создана!')
print(f'📏 Размер начального наблюдения: {initial_observation.shape}')
initial_observation, info

(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 {})

## 🎮 Выполнение шага симуляции

Применяем управляющее воздействие (отклонение руля высоты) и наблюдаем реакцию системы:

In [4]:
# Применение управляющего воздействия
control_action = np.array([1])  # Отклонение руля высоты (elevator)

# Выполнение одного шага симуляции
observation, reward, terminated, truncated, info = env.step(control_action)

print(f'🎯 Управляющее воздействие: {control_action[0]}')
print(f'🏆 Награда: {reward}')
print(f'🔚 Завершено: {terminated}')
print(f'⏰ Прервано по времени: {truncated}')

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


## 📊 Анализ результатов

Рассмотрим полученные наблюдения и награды после выполнения шага:

In [5]:
# Текущее состояние системы после применения управления
print('🔍 Состояние системы F-16:')
print(f'   • Угол тангажа (theta): {observation[0][0]:.6f} рад')
print(f'   • Угол атаки (alpha): {observation[1][0]:.6f} рад')
print(f'   • Угловая скорость тангажа (q): {observation[2][0]:.6f} рад/с')
print(f'   • Высота (altitude): {observation[3][0]:.6f} м')
print(f'   • Скорость (velocity): {observation[4][0]:.6f} м/с')

observation

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

### 🎯 Анализ награды

Оценим качество управления через полученную награду:

In [6]:
# Анализ полученной награды
print('🎯 Анализ награды:')
print(f'   • Текущая награда: {reward:.6f}')

if reward >= 0:
    print('   ✅ Положительная награда - хорошее управление!')
elif reward < 0:
    print('   ⚠️  Отрицательная награда - требуется улучшение управления')

print('\n📈 Интерпретация:')
print('   • Награда отражает качество отслеживания опорного сигнала')
print('   • Чем ближе к нулю отклонение, тем выше награда')
print('   • Отрицательные значения указывают на ошибки управления')

reward

1

### 🏁 Завершение эпизода

Проверим статус завершения симуляции:

In [7]:
# Проверка статуса завершения эпизода
done = terminated or truncated
print('🏁 Статус симуляции:')
if done:
    print('   ✅ Эпизод завершен')
    print('   📊 Можно анализировать результаты или начать новый эпизод')
else:
    print('   🔄 Эпизод продолжается')
    print('   ➡️  Можно выполнить следующий шаг симуляции')

print(f'\n🔢 Числовое значение done: {done}')
print('   • True = эпизод завершен')
print('   • False = эпизод продолжается')

done

False

---

## 🎓 Заключение

В этом примере мы продемонстрировали основы работы с **линейной продольной моделью F-16** в TensorAeroSpace:

### ✅ Что мы изучили:
- 🛠️ **Создание среды**: Инициализация LinearLongitudinalF16Environment
- 🎯 **Настройка параметров**: Конфигурация симуляции и опорных сигналов
- 🚀 **Выполнение шагов**: Применение управляющих воздействий
- 📊 **Анализ результатов**: Интерпретация состояний, наград и статусов

### 🔄 Следующие шаги:
- 🎮 **Попробуйте разные управляющие воздействия** для изучения поведения системы
- 📈 **Реализуйте контроллер** для автоматического управления
- 🧠 **Обучите агента обучения с подкреплением** для оптимального управления
- 📋 **Изучите другие среды** в TensorAeroSpace

### 📚 Полезные ресурсы:
- [Документация TensorAeroSpace](https://tensoraerospace.readthedocs.io/)
- [Примеры использования](https://github.com/TensorAeroSpace/TensorAeroSpace/tree/main/examples)
- [Теория управления летательными аппаратами](https://en.wikipedia.org/wiki/Aircraft_flight_dynamics)

**Удачи в изучении аэрокосмических систем! 🚀✈️**